<a href="https://colab.research.google.com/github/martinalegre77/python/blob/main/app_presupuesto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
class Category:

  def __init__(self, name):
    self.name = name.capitalize()
    self.ledger = []
    self.funds = 0
    self.spend = 0

  def deposit(self, amount, description=""):
    self.ledger.append({"amount": amount,
                        "description": description})
    self.funds += amount

  def withdraw(self, amount, description=""):
    if self.check_funds(amount):
      self.ledger.append({"amount": -amount,
                          "description": description})
      self.funds -= amount
      self.spend += amount
      return True
    else:
      return False

  def get_balance(self):
    return (f'{self.funds:.2f}\n')

  def transfer(self, amount, category):
    if self.check_funds(amount):
      self.ledger.append({"amount": -amount,
                          "description": f"Transfer to {category.name}"})
      category.deposit(amount, f"Transfer from {self.name}")
      self.funds -= amount
      return True
    else:
      return False

  def check_funds(self, amount):
    return amount <= self.funds

  def __str__(self):
    head = int((30 - len(self.name)) / 2)
    if len(self.name) % 2 == 0:
      titulo = "*" * head + self.name + "*" * head
    else:
      titulo = "*" * head + self.name + "*" * (head + 1)
    contenido = ""
    for i in range(len(self.ledger)):
      valor = float(self.ledger[i]["amount"])
      concepto = self.ledger[i]["description"].capitalize()
      contenido += f"{concepto[:23]:23}{valor:>7.2f}\n"
    total = f"Total: {self.funds:.2f}"
    return (f'{titulo}\n' + f'{contenido}' + f'{total}\n')

# crear gráfico de gastos
def create_spend_chart(categories):
  concep = len(categories)
  ref = [False] * concep
  # titulo
  titulo = 'Percentage spent by category'
  # tabla de porcentajes
  contenido = ""
  for x in range(100, -1, -10):
    contenido += f'{x:>3}|'
    for z in range(concep):
      if categories[z].spend > x and categories[z].spend < x+10:
        contenido += ' o '
        ref[z] = True
      elif ref[z]:
        contenido += ' o '
      else:
        contenido += '   '
    contenido += '\n'
  # separador
  sep = '    ' + ('---' * concep) + '-'
  # palabras
  li_pal = []
  palabras = ""
  mas_larga = 0
  for z in range(concep):
    li_pal.append(categories[z].name)
    if len(categories[z].name) > mas_larga:
      mas_larga = len(categories[z].name)
  for x in range(mas_larga):
    palabras += '    '
    for i in range(concep):
      if x < len(li_pal[i]):
        palabras += f' {li_pal[i][x]} '
      else:
        palabras += '   '
    palabras += '\n'

  return (f'{titulo}\n'
          + f'{contenido}'
          + f'{sep}\n'
          + f'{palabras}')

# TEST

In [108]:
class UnitTests():
    maxDiff = None
    def setUp(self):
        self.food = Category("Food")
        self.entertainment = Category("Entertainment")
        self.business = Category("Business")

    def test_deposit(self):
        self.food.deposit(900, "deposit")
        actual = self.food.ledger[0]
        expected = {"amount": 900, "description": "deposit"}
        self.assertEqual(actual, expected, 'Expected `deposit` method to create a specific object in the ledger instance variable.')

    def test_deposit_no_description(self):
        self.food.deposit(45.56)
        actual = self.food.ledger[0]
        expected = {"amount": 45.56, "description": ""}
        self.assertEqual(actual, expected, 'Expected calling `deposit` method with no description to create a blank description.')

    def test_withdraw(self):
        self.food.deposit(900, "deposit")
        self.food.withdraw(45.67, "milk, cereal, eggs, bacon, bread")
        actual = self.food.ledger[1]
        expected = {"amount": -45.67, "description": "milk, cereal, eggs, bacon, bread"}
        self.assertEqual(actual, expected, 'Expected `withdraw` method to create a specific object in the ledger instance variable.')

    def test_withdraw_no_description(self):
        self.food.deposit(900, "deposit")
        good_withdraw = self.food.withdraw(45.67)
        actual = self.food.ledger[1]
        expected = {"amount": -45.67, "description": ""}
        self.assertEqual(actual, expected, 'Expected `withdraw` method with no description to create a blank description.')
        self.assertEqual(good_withdraw, True, 'Expected `withdraw` method to return `True`.')

    def test_get_balance(self):
        self.food.deposit(900, "deposit")
        self.food.withdraw(45.67, "milk, cereal, eggs, bacon, bread")
        actual = self.food.get_balance()
        expected = 854.33
        self.assertEqual(actual, expected, 'Expected balance to be 854.33')

    def test_transfer(self):
        self.food.deposit(900, "deposit")
        self.food.withdraw(45.67, "milk, cereal, eggs, bacon, bread")
        transfer_amount = 20
        food_balance_before = self.food.get_balance()
        entertainment_balance_before = self.entertainment.get_balance()
        good_transfer = self.food.transfer(transfer_amount, self.entertainment)
        food_balance_after = self.food.get_balance()
        entertainment_balance_after = self.entertainment.get_balance()
        actual = self.food.ledger[2]
        expected = {"amount": -transfer_amount, "description": "Transfer to Entertainment"}
        self.assertEqual(actual, expected, 'Expected `transfer` method to create a specific ledger item in food object.')
        self.assertEqual(good_transfer, True, 'Expected `transfer` method to return `True`.')
        self.assertEqual(food_balance_before - food_balance_after, transfer_amount, 'Expected `transfer` method to reduce balance in food object.')
        self.assertEqual(entertainment_balance_after - entertainment_balance_before, transfer_amount, 'Expected `transfer` method to increase balance in entertainment object.')
        actual = self.entertainment.ledger[0]
        expected = {"amount": transfer_amount, "description": "Transfer from Food"}
        self.assertEqual(actual, expected, 'Expected `transfer` method to create a specific ledger item in entertainment object.')

    def test_check_funds(self):
        self.food.deposit(10, "deposit")
        actual = self.food.check_funds(20)
        expected = False
        self.assertEqual(actual, expected, 'Expected `check_funds` method to be False')
        actual = self.food.check_funds(10)
        expected = True
        self.assertEqual(actual, expected, 'Expected `check_funds` method to be True')

    def test_withdraw_no_funds(self):
        self.food.deposit(100, "deposit")
        good_withdraw = self.food.withdraw(100.10)
        self.assertEqual(good_withdraw, False, 'Expected `withdraw` method to return `False`.')

    def test_transfer_no_funds(self):
        self.food.deposit(100, "deposit")
        good_transfer = self.food.transfer(200, self.entertainment)
        self.assertEqual(good_transfer, False, 'Expected `transfer` method to return `False`.')

    def test_to_string(self):
        self.food.deposit(900, "deposit")
        self.food.withdraw(45.67, "milk, cereal, eggs, bacon, bread")
        self.food.transfer(20, self.entertainment)
        actual = str(self.food)
        expected = f"*************Food*************\ndeposit                 900.00\nmilk, cereal, eggs, bac -45.67\nTransfer to Entertainme -20.00\nTotal: 834.33"
        self.assertEqual(actual, expected, 'Expected different string representation of object.')

    def test_create_spend_chart(self):
        self.food.deposit(900, "deposit")
        self.entertainment.deposit(900, "deposit")
        self.business.deposit(900, "deposit")
        self.food.withdraw(105.55)
        self.entertainment.withdraw(33.40)
        self.business.withdraw(10.99)
        actual = create_spend_chart([self.business, self.food, self.entertainment])
        expected = "Percentage spent by category\n100|          \n 90|          \n 80|          \n 70|    o     \n 60|    o     \n 50|    o     \n 40|    o     \n 30|    o     \n 20|    o  o  \n 10|    o  o  \n  0| o  o  o  \n    ----------\n     B  F  E  \n     u  o  n  \n     s  o  t  \n     i  d  e  \n     n     r  \n     e     t  \n     s     a  \n     s     i  \n           n  \n           m  \n           e  \n           n  \n           t  "
        self.assertEqual(actual, expected, 'Expected different chart representation. Check that all spacing is exact.')

# if __name__ == "__main__":
#     unittest.main()

# MAIN

In [109]:
food = Category("Food")
food.deposit(1000, "initial deposit")
food.withdraw(10.15, "groceries")
food.withdraw(15.89, "restaurant and more food for dessert")
print(food.get_balance())
clothing = Category("Clothing")
food.transfer(50, clothing)
clothing.withdraw(25.55)
clothing.withdraw(100)
auto = Category("Auto")
auto.deposit(1000, "initial deposit")
auto.withdraw(15)

print(food)
print(clothing)

print(create_spend_chart([food, clothing, auto]))

# Run unit tests automatically
# main(module='test_module', exit=False)

973.96

*************Food*************
Initial deposit        1000.00
Groceries               -10.15
Restaurant and more foo -15.89
Transfer to clothing    -50.00
Total: 923.96

***********Clothing***********
Transfer from food       50.00
                        -25.55
Total: 24.45

Percentage spent by category
100|         
 90|         
 80|         
 70|         
 60|         
 50|         
 40|         
 30|         
 20| o  o    
 10| o  o  o 
  0| o  o  o 
    ----------
     F  C  A 
     o  l  u 
     o  o  t 
     d  t  o 
        h    
        i    
        n    
        g    

